## Homework 02 - Samuel Zeleke - CS344

#### Exercise 1
Build a spam filter based on Paul Graham’s A Plan for Spam. You’ll find a sketch of his statistical algorithm early in 
the article (roughly one-fifth of the way through the article).

Include in your solution, as one test case, the probability tables for the words in the following hard-coded SPAM/HAM 
corpus (and only this corpus) using a minimum count threshold of 1 (rather than the 5 used in the algorithm):

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]

ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

ANSWER:

In [1]:
# Utility variables
import pprint

prettier = pprint.PrettyPrinter()

probTable = dict()

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def spamFilterModel(spam, ham):
    wordStats = dict()

    # count word in spam
    for document in spam:
        for word in document:
            if word in wordStats.keys():
                wordStats[word]["spam"] += 1
            else:
                wordStats[word] = {"spam": 1, "ham": 0}

    # count word in ham
    for document in ham:
        for word in document:
            if word in wordStats.keys():
                wordStats[word]["ham"] += 1
            else:
                wordStats[word] = {"ham": 1, "spam": 0}

    # calculate the probabilities
    returnContainer = dict()

    for word in wordStats:
        # if the word appears more than once in documents, calculate the probabilities
        if wordStats[word]["spam"] + wordStats[word]["ham"] > 1:
            # if the count for the word is more than the number of documents, set the prob to 0.99. Otherwise, find the 
            # word count-to-number of documents ratio 
            statSpam = 0.99 if wordStats[word]["spam"] / len(spam) > 0.99 else wordStats[word]["spam"] / len(spam)
            statHam = 0.99 if 2*wordStats[word]["ham"]/len(ham) > 0.99 else 2*wordStats[word]["ham"]/len(ham)
            # 
            prob = statSpam / (statHam + statSpam)

            returnContainer[word] = prob if prob > 0.01 else 0.01

    return returnContainer

prettier.pprint(spamFilterModel(spam_corpus, ham_corpus))

def spamIdentifier(model, string):
    probForSpam = 1
    probAgainstSpam = 1
    for word in string.split(" "):
        # if the word was in the "training" corpus then multiply with its probability values.
        # otherwise, use the likelihood of being spam to 0.4
        if word in model.keys():
            probForSpam *= model[word]
            probAgainstSpam *= (1 - model[word])
        else:
            probForSpam *= 0.4
            probAgainstSpam *= 0.6
    return probForSpam / (probForSpam + probAgainstSpam)
testStrings = [
    "I am a spam email that uses a lot of i i i i s.",
    "do I take CS344?",
    "do i take CS344?",
    "i i i i i notSpam",
]

for string in testStrings:
    print("Testing on string: \"", string, "\"\n  Probability of being Spam is: ", 
          spamIdentifier(spamFilterModel(spam_corpus, ham_corpus), string))

{'I': 1.0,
 'am': 1.0,
 'do': 0.33557046979865773,
 'i': 0.01,
 'like': 0.33557046979865773,
 'spam': 1.0}
Testing on string: " I am a spam email that uses a lot of i i i i s. "
  Probability of being Spam is:  1.0
Testing on string: " do I take CS344? "
  Probability of being Spam is:  1.0
Testing on string: " do i take CS344? "
  Probability of being Spam is:  0.002262213123098327
Testing on string: " i i i i i notSpam "
  Probability of being Spam is:  7.010238084930902e-11


Graham argues that this is a Baysian approach to SPAM. What makes it Bayesian?

ANSWER:
    The approach evaluates an email based on the conditional probability of the email being spam given certain words. It
     does not need a complete joint probability table for every possible combination of words. 
    
#### Exercise 2

Do the following exercises based on the Bayesian network shown in Figure 14.12a:

A. Implement the network using the AIMA Python tools.

ANSWER:

In [2]:
from probability import BayesNet, enumeration_ask

# Utility variables
T, F = True, False
grass = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F:0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0}),
    ])
# P(Cloudy)
print("i.   P(Cloudy):                               ", enumeration_ask('Cloudy', dict(), grass).show_approx())
# P(Cloudy | ¬WetGrass)
print("ii.  P(Sprinkler | Cloudy):                   ", enumeration_ask('Sprinkler', dict(Cloudy=T), grass).show_approx())
# calculating P(Cloudy | Sprinkler ⋂ ¬Rain)
print("iii. P(Cloudy | Sprinkler ⋂ ¬Rain):           ", enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), grass).show_approx())
# P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain)
print("iv.  P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain): ", enumeration_ask('WetGrass', dict(Sprinkler=T, Rain=T, Cloudy=T), grass).show_approx())
# P(Cloudy | ¬WetGrass)
print("v.   P(Cloudy | ¬WetGrass):                   ", enumeration_ask('Cloudy', dict(WetGrass=F), grass).show_approx())

i.   P(Cloudy):                                False: 0.5, True: 0.5
ii.  P(Sprinkler | Cloudy):                    False: 0.9, True: 0.1
iii. P(Cloudy | Sprinkler ⋂ ¬Rain):            False: 0.952, True: 0.0476
iv.  P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain):  False: 0.01, True: 0.99
v.   P(Cloudy | ¬WetGrass):                    False: 0.639, True: 0.361


B. Compute the number of independent values in the full joint probability distribution for this domain. Assume that no 
conditional independence relations are known to hold between these values.

ANSWER: There are 3 independent variables and 8 possible combinations (values).

C. Compute the number of independent values in the Bayesian network for this domain. Assume the conditional independence 
relations implied by the Bayes network.

ANSWER: There's one independent variable (Cloudiness).

D. Compute probabilities for the following:

ANSWER:

    i. P(Cloudy) = <0.5, 0.5>
    ii. P(Sprinkler | Cloudy) = <0.1, 0.9>
    iii. P(Cloudy | Sprinkler ⋂ ¬Rain) = <0.0476, 0.952>
    
    P(Cloudy | Sprinkler ⋂ ¬Rain) = α<P(Cloudy ⋂ Sprinkler ⋂ ¬Rain), P(Cloudy ⋂ Sprinkler ⋂ ¬Rain)>
                   P(Cloudy ⋂ Sprinkler ⋂ ¬Rain) = P(Cloudy)*P(Sprinker | Cloudy)*P(Cloudy)*P(¬Rain | Cloudy)*P(Cloudy)
                                                 = 0.5 * 0.1 * 0.5 * 0.2 * 0.5 
                                                 = 0.0025
                   P(¬Cloudy ⋂ Sprinkler ⋂ ¬Rain) = P(¬Cloudy)*P(Sprinker | ¬Cloudy)*P(¬Cloudy)*P(¬Rain | ¬Cloudy)*P(¬Cloudy)
                                                 = 0.5 * 0.5 * 0.5 * 0.8 * 0.5 
                                                 = 0.05
    P(Cloudy | Sprinkler ⋂ ¬Rain) = α<0.0025, 0.05>, α = 1/0.0525
                                  = <0.04762, 0.95238>
                                  
    iv. P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain)
    
        P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain) = α<P(WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain), P(¬WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain)>
            P(WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain) = 
                P(WetGrass | Sprinkler ⋂ Rain) * P(Sprinkler | Cloudy) * 
                P(Cloudy) * P(Rain | Cloudy) *P(Cloudy) * P(Cloudy)
            P(WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain) = 0.99 * 0.1 * 0.5 * 0.8 * 0.5 * 0.5
                                                    = 0.0099
            P(¬WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain) = 
                P(¬WetGrass | Sprinkler ⋂ Rain) * P(Sprinkler | Cloudy) * 
                P(Cloudy) * P(Rain | Cloudy) *P(Cloudy) * P(Cloudy)
            P(WetGrass ⋂ Cloudy ⋂ Sprinkler ⋂ Rain) = 0.01 * 0.1 * 0.5 * 0.8 * 0.5 * 0.5
                                                    = 0.0001
        P(WetGrass | Cloudy ⋂ Sprinkler ⋂ Rain) = α<0.0099, 0.0001>, α = 1/0.01
                                                = <0.99, 0.01>                                       
            
    v. P(Cloudy | ¬WetGrass)
        P(Cloudy | ¬WetGrass) = α<P(Cloudy ⋂ ¬WetGrass), P(¬Cloudy ⋂ ¬WetGrass)>
            P(Cloudy ⋂ ¬WetGrass) = 
                P(Cloudy) * (P(¬WetGrass | Sprinkler ⋂ Rain) * P(Sprinkler) * P(Rain) + 
                             P(¬WetGrass | ¬Sprinkler ⋂ Rain) * P(¬Sprinkler) * P(Rain) +
                             P(¬WetGrass | Sprinkler ⋂ ¬Rain) * P(Sprinkler) * P(¬Rain) + 
                             P(¬WetGrass | ¬Sprinkler ⋂ ¬Rain) * P(¬Sprinkler) * P(¬Rain)
                             )
              P(Sprinkler) = P(Sprinkler | Cloudy) * P(Cloudy)
                           = 0.1*0.5
                           = 0.05
              P(¬Sprinkler) = P(¬Sprinkler | Cloudy) * P(Cloudy)
                            = 0.9*0.5
                            = 0.45
              P(Rain) = P(Rain | Cloudy) * P(Cloudy)
                      = 0.8 * 0.5
                      = 0.4
              P(¬Rain) = P(¬Rain | Cloudy) * P(Cloudy)
                       = 0.2*0.5
                       = 0.1
                       
            P(Cloudy ⋂ ¬WetGrass) = 0.5 * (0.01 * 0.05 * 0.4 + 0.1 * 0.45 * 0.4 + 0.1 * 0.05 * 0.1 + 1 * 0.45 * 0.1)
                                  = 0.03185
            ----------------------------------------------------------------------------------------------------
            P(¬Cloudy ⋂ ¬WetGrass) = 
                P(¬Cloudy) * (P(¬WetGrass | Sprinkler ⋂ Rain) * P(Sprinkler) * P(Rain) + 
                             P(¬WetGrass | ¬Sprinkler ⋂ Rain) * P(¬Sprinkler) * P(Rain) +
                             P(¬WetGrass | Sprinkler ⋂ ¬Rain) * P(Sprinkler) * P(¬Rain) + 
                             P(¬WetGrass | ¬Sprinkler ⋂ ¬Rain) * P(¬Sprinkler) * P(¬Rain)
                             )
              P(Sprinkler) = P(Sprinkler | ¬Cloudy) * P(¬Cloudy)
                           = 0.5*0.5
                           = 0.25
              P(¬Sprinkler) = P(¬Sprinkler | ¬Cloudy) * P(¬Cloudy)
                            = 0.5*0.5
                            = 0.25
              P(Rain) = P(Rain | ¬Cloudy) * P(¬Cloudy)
                      = 0.2 * 0.5
                      = 0.1
              P(¬Rain) = P(¬Rain | ¬Cloudy) * P(¬Cloudy)
                       = 0.8*0.5
                       = 0.4
                       
            P(¬Cloudy ⋂ ¬WetGrass) = 0.5 * (0.01 * 0.25 * 0.1 + 0.1 * 0.25 * 0.1 + 0.1 * 0.25 * 0.4 + 1 * 0.25 * 0.4)
                                   = 0.056375
        P(Cloudy | ¬WetGrass) = α<P(Cloudy ⋂ ¬WetGrass), P(¬Cloudy ⋂ ¬WetGrass)>
                              = α<0.03185, 0.056375>, α= 1 /(0.03185 + 0.056375)
                              = <0.361, 0.639>